## for every user we want the weight of each of the genres for the movies they have watched. the recent movies will get more weight.


### * we are gonna do this for the training set 

In [77]:
import pandas as pd
import numpy as np

In [78]:
import os

print(os.getcwd())

/Users/tahsinalamgirkheya/Desktop/work/reranking_fairnes/user


In [79]:
rslt_file = '../data/ml-100k/indexed_interactions.csv'
df = pd.read_csv(rslt_file, sep='\t', names=["userID", "itemID", "rating", "timestamp"],
    header=None,)
print(f'shape: {df.shape}')


shape: (99287, 4)


In [80]:
movie = 'ml-1m final/i_id_mapping.csv'
df_movie = pd.read_csv(movie, sep='\t')
print(f'shape: {df_movie.shape}')
df_movie[:4]

FileNotFoundError: [Errno 2] No such file or directory: 'ml-1m final/i_id_mapping.csv'

In [81]:
df_m = pd.read_csv(
    "../data/ml-100k/u.item",
    sep="|",
    names=[
        "movieID",
        "Name",
        "Date",
        "Video_Date",
        "IMDB_URL",
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ],
    header=None,
    encoding="latin-1",
)
print(df_m.shape)
df_m = df_m[
    [
        "movieID",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ]
]

df_movies_mapped = pd.read_csv(
    "../data/ml-100k/i_id_mapping.csv",
    sep="\t",
    names=["movieID", "itemID"],
    header=None,
    encoding="latin-1",
)
movies = pd.merge(df_m, df_movies_mapped, how="inner", on="movieID")
movies

(1682, 24)


,movieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,24
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,147
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,233
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,47
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,1592,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1305
1345,1597,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1324
1346,1598,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1319
1347,1615,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1341


In [82]:
unique_genres = ['Action', 'Thriller', 'Romance', 'Western', "Children's", 'Mystery', 'Fantasy', 'Film-Noir', 'Documentary', 'Comedy', 'Adventure', 'Sci-Fi', 'Horror', 'Crime', 'Musical', 'War', 'Animation', 'Drama']

# for genre in unique_genres:
#     df_movie[genre] = 0
# for index, row in df_movie.iterrows():
#     genres = row['Genre'].split('|')
#     for genre in genres:
#         df_movie.at[index, genre] = 1
df_movie = movies.sort_values(by="itemID")
df_movie


,movieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID
240,242,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
300,302,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1
375,377,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2
50,51,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,3
344,346,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,1335,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1344
1192,1251,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1345
1176,1233,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1346
1261,1380,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1347


In [83]:

df_movie[unique_genres] = df_movie[unique_genres].div(df_movie[unique_genres].sum(axis=1), axis=0)

In [84]:
movie_interactions = pd.merge(df, df_movie, on ="itemID", how="inner")
movie_interactions


,userID,itemID,rating,timestamp,movieID,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,3,881250949,242,0.0,0.0,0.0,0.0,1.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00
1,1,1,3,891717742,302,0.0,0.0,0.0,0.0,0.0,...,0.0,0.25,0.0,0.0,0.25,0.00,0.0,0.25,0.00,0.00
2,2,2,1,878887116,377,0.0,0.0,0.0,0.5,0.5,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00
3,3,3,2,880606923,51,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.25,0.0,0.00,0.25,0.25
4,4,4,1,886397596,346,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99282,875,173,3,880175444,476,0.0,0.0,0.0,0.0,1.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00
99283,708,247,5,879795543,204,0.0,0.0,0.0,0.0,0.5,...,0.0,0.00,0.0,0.0,0.00,0.00,0.5,0.00,0.00,0.00
99284,37,982,1,874795795,1090,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,1.00,0.00,0.00
99285,58,442,2,882399156,225,0.0,0.0,0.0,0.5,0.5,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00


In [85]:
movie_interactions[movie_interactions["userID"]==4]

,userID,itemID,rating,timestamp,movieID,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
4,4,4,1,886397596,346,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
105,4,98,5,886397722,328,0.250000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.25,0.250000,0.000000,0.250000,0.000000,0.0
3182,4,51,5,886397723,322,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.50,0.000000,0.000000,0.500000,0.000000,0.0
5067,4,60,3,886397510,288,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.500000,0.0,0.00,0.000000,0.000000,0.500000,0.000000,0.0
8815,4,157,4,886397562,258,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.000000,0.500000,0.000000,0.000000,0.0
9171,4,252,4,886397882,343,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.333333,0.0,0.00,0.000000,0.333333,0.000000,0.000000,0.0
11545,4,645,5,886397723,300,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.500000,0.000000,0.0
13823,4,964,4,886397905,894,0.000000,0.0,0.0,0.5,0.5,...,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0
14108,4,144,5,886397722,323,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.500000,0.000000,0.0
17479,4,175,2,886397751,748,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.333333,0.000000,0.333333,0.000000,0.0


In [88]:
movie_interactions[movie_interactions["userID"]==4]

,userID,itemID,rating,timestamp,movieID,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
4,4,4,1,886397596,346,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
105,4,98,5,886397722,328,2.215994e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,221599430.5,2.215994e+08,0.000000e+00,2.215994e+08,0.0,0.0
3182,4,51,5,886397723,322,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,443198861.5,0.000000e+00,0.000000e+00,4.431989e+08,0.0,0.0
5067,4,60,3,886397510,288,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,4.431988e+08,0.0,0.0,0.000000e+00,0.000000e+00,4.431988e+08,0.0,0.0
8815,4,157,4,886397562,258,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,4.431988e+08,0.000000e+00,0.0,0.0
9171,4,252,4,886397882,343,2.954660e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,2.954660e+08,0.0,0.0,0.000000e+00,2.954660e+08,0.000000e+00,0.0,0.0
11545,4,645,5,886397723,300,4.431989e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,4.431989e+08,0.0,0.0
13823,4,964,4,886397905,894,0.000000e+00,0.0,0.0,443198952.5,443198952.5,...,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
14108,4,144,5,886397722,323,4.431989e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,4.431989e+08,0.0,0.0
17479,4,175,2,886397751,748,2.954659e+08,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,2.954659e+08,0.000000e+00,2.954659e+08,0.0,0.0


In [86]:
movie_interactions[unique_genres] = movie_interactions[unique_genres].multiply(movie_interactions["timestamp"], axis=0)


In [87]:
movie_interactions

,userID,itemID,rating,timestamp,movieID,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,3,881250949,242,0.0,0.0,0.0,0.0,881250949.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
1,1,1,3,891717742,302,0.0,0.0,0.0,0.0,0.0,...,0.0,222929435.5,0.0,0.0,222929435.5,0.000000e+00,0.0,222929435.5,0.000000e+00,0.000000e+00
2,2,2,1,878887116,377,0.0,0.0,0.0,439443558.0,439443558.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
3,3,3,2,880606923,51,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.201517e+08,0.0,0.0,2.201517e+08,2.201517e+08
4,4,4,1,886397596,346,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99282,875,173,3,880175444,476,0.0,0.0,0.0,0.0,880175444.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
99283,708,247,5,879795543,204,0.0,0.0,0.0,0.0,439897771.5,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,439897771.5,0.0,0.000000e+00,0.000000e+00
99284,37,982,1,874795795,1090,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,874795795.0,0.000000e+00,0.000000e+00
99285,58,442,2,882399156,225,0.0,0.0,0.0,441199578.0,441199578.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00


In [89]:
summed_genre = movie_interactions.groupby("userID")[unique_genres].sum().reset_index()
summed_genre

,userID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,Film-Noir,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,0,6.609389e+08,0.000000e+00,4.112508e+09,0.000000e+00,7.343767e+08,0.000000e+00,2.203130e+08,0.000000e+00,8.812511e+08,1.982816e+10,9.546895e+08,4.406259e+08,0.000000e+00,0.000000e+00,0.000000e+00,5.140634e+08,0.000000e+00,6.021886e+09
1,1,1.293266e+10,2.241718e+10,2.503263e+09,2.203907e+09,3.331990e+09,3.529847e+09,4.688130e+08,7.359257e+08,0.000000e+00,6.000171e+09,4.299183e+09,2.567341e+09,8.790231e+08,4.340082e+09,1.098781e+09,5.860155e+08,1.274586e+09,1.201259e+10
2,2,2.131303e+10,7.529140e+09,5.991081e+09,1.318331e+09,3.515549e+09,8.788876e+08,2.197218e+08,4.394438e+08,0.000000e+00,3.985752e+10,9.711709e+09,7.778154e+09,1.230442e+09,2.153275e+09,1.054664e+09,4.599511e+09,0.000000e+00,4.907121e+09
3,3,1.604170e+10,1.169737e+10,1.943203e+10,1.100759e+09,2.715203e+09,1.981359e+09,5.136870e+08,8.806044e+08,8.806055e+08,6.225884e+10,5.811998e+09,7.059516e+09,3.214214e+09,2.817933e+09,3.302273e+09,6.883398e+09,3.478390e+09,5.863364e+10
4,4,2.289861e+09,3.914923e+09,1.403463e+09,0.000000e+00,8.863979e+08,6.647983e+08,0.000000e+00,0.000000e+00,0.000000e+00,3.545591e+09,0.000000e+00,7.386647e+08,7.386647e+08,4.431988e+08,0.000000e+00,7.386647e+08,0.000000e+00,2.363727e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,938,6.308539e+09,4.768083e+09,4.181241e+09,0.000000e+00,1.760522e+08,2.934205e+08,2.200655e+08,0.000000e+00,0.000000e+00,8.538539e+09,3.110256e+09,2.420719e+09,8.802620e+08,1.026972e+09,1.760522e+08,1.026971e+09,1.760522e+08,9.829584e+09
939,939,8.779643e+09,1.071589e+10,1.166185e+10,0.000000e+00,1.625859e+09,2.882204e+09,0.000000e+00,0.000000e+00,1.773665e+09,2.993060e+10,3.680357e+09,4.197673e+09,1.182443e+09,5.468802e+09,1.182443e+09,1.167663e+09,2.956108e+08,3.872502e+10
940,940,6.332653e+09,3.518140e+09,5.966179e+09,0.000000e+00,8.795350e+08,2.931783e+08,2.198838e+08,0.000000e+00,0.000000e+00,1.385268e+10,2.887807e+09,2.667924e+09,4.397670e+08,5.863565e+08,1.319303e+09,2.594628e+09,7.329458e+08,1.311973e+10
941,941,1.473700e+09,4.126360e+09,1.547385e+09,0.000000e+00,0.000000e+00,1.694755e+09,0.000000e+00,5.157950e+08,0.000000e+00,1.326330e+09,4.421100e+08,7.368500e+08,9.579049e+08,1.473700e+09,0.000000e+00,5.157950e+08,0.000000e+00,7.294814e+09


In [94]:
734376700/34368817581

0.02136752881501466

In [90]:
user_timestamp_sum=movie_interactions.groupby('userID')['timestamp'].sum()
summed_genre['user_timestamp_sum'] = summed_genre['userID'].map(user_timestamp_sum)


In [91]:
summed_genre[unique_genres] = summed_genre[unique_genres].div(summed_genre["user_timestamp_sum"], axis=0)
summed_genre


,userID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,Film-Noir,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama,user_timestamp_sum
0,0,0.019231,0.000000,0.119658,0.000000,0.021368,0.000000,0.006410,0.000000,0.025641,0.576923,0.027778,0.012821,0.000000,0.000000,0.000000,0.014957,0.000000,0.175214,34368817581
1,1,0.159306,0.276137,0.030835,0.027148,0.041044,0.043481,0.005775,0.009065,0.000000,0.073911,0.052958,0.031625,0.010828,0.053462,0.013535,0.007219,0.015700,0.147972,81181348894
2,2,0.189453,0.066927,0.053255,0.011719,0.031250,0.007813,0.001953,0.003906,0.000000,0.354297,0.086328,0.069141,0.010937,0.019141,0.009375,0.040885,0.000000,0.043620,112497575458
3,3,0.076864,0.056048,0.093108,0.005274,0.013010,0.009494,0.002461,0.004219,0.004219,0.298312,0.027848,0.033826,0.015401,0.013502,0.015823,0.032982,0.016667,0.280942,208703517034
4,4,0.129167,0.220833,0.079167,0.000000,0.050000,0.037500,0.000000,0.000000,0.000000,0.200000,0.000000,0.041667,0.041667,0.025000,0.000000,0.041667,0.000000,0.133333,17727953698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,938,0.146258,0.110544,0.096939,0.000000,0.004082,0.006803,0.005102,0.000000,0.000000,0.197959,0.072109,0.056122,0.020408,0.023810,0.004082,0.023810,0.004082,0.227891,43132807063
939,939,0.071223,0.086930,0.094604,0.000000,0.013189,0.023381,0.000000,0.000000,0.014388,0.242806,0.029856,0.034053,0.009592,0.044365,0.009592,0.009472,0.002398,0.314149,123269730515
940,940,0.114286,0.063492,0.107672,0.000000,0.015873,0.005291,0.003968,0.000000,0.000000,0.250000,0.052116,0.048148,0.007936,0.010582,0.023810,0.046825,0.013228,0.236772,55410714285
941,941,0.066667,0.186667,0.070000,0.000000,0.000000,0.076667,0.000000,0.023333,0.000000,0.060000,0.020000,0.033333,0.043333,0.066667,0.000000,0.023333,0.000000,0.330000,22105499005


In [92]:
summed_genre

,userID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,Film-Noir,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama,user_timestamp_sum
0,0,0.019231,0.000000,0.119658,0.000000,0.021368,0.000000,0.006410,0.000000,0.025641,0.576923,0.027778,0.012821,0.000000,0.000000,0.000000,0.014957,0.000000,0.175214,34368817581
1,1,0.159306,0.276137,0.030835,0.027148,0.041044,0.043481,0.005775,0.009065,0.000000,0.073911,0.052958,0.031625,0.010828,0.053462,0.013535,0.007219,0.015700,0.147972,81181348894
2,2,0.189453,0.066927,0.053255,0.011719,0.031250,0.007813,0.001953,0.003906,0.000000,0.354297,0.086328,0.069141,0.010937,0.019141,0.009375,0.040885,0.000000,0.043620,112497575458
3,3,0.076864,0.056048,0.093108,0.005274,0.013010,0.009494,0.002461,0.004219,0.004219,0.298312,0.027848,0.033826,0.015401,0.013502,0.015823,0.032982,0.016667,0.280942,208703517034
4,4,0.129167,0.220833,0.079167,0.000000,0.050000,0.037500,0.000000,0.000000,0.000000,0.200000,0.000000,0.041667,0.041667,0.025000,0.000000,0.041667,0.000000,0.133333,17727953698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,938,0.146258,0.110544,0.096939,0.000000,0.004082,0.006803,0.005102,0.000000,0.000000,0.197959,0.072109,0.056122,0.020408,0.023810,0.004082,0.023810,0.004082,0.227891,43132807063
939,939,0.071223,0.086930,0.094604,0.000000,0.013189,0.023381,0.000000,0.000000,0.014388,0.242806,0.029856,0.034053,0.009592,0.044365,0.009592,0.009472,0.002398,0.314149,123269730515
940,940,0.114286,0.063492,0.107672,0.000000,0.015873,0.005291,0.003968,0.000000,0.000000,0.250000,0.052116,0.048148,0.007936,0.010582,0.023810,0.046825,0.013228,0.236772,55410714285
941,941,0.066667,0.186667,0.070000,0.000000,0.000000,0.076667,0.000000,0.023333,0.000000,0.060000,0.020000,0.033333,0.043333,0.066667,0.000000,0.023333,0.000000,0.330000,22105499005


In [96]:
summed_genre.to_csv("pgui.csv", sep="\t", index=False)